In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Distributed Computing
<!-- requirement: small_data/gutenberg -->
<!-- requirement: small_data/fha_by_tract.csv -->
<!-- requirement: scripts/mapper.py -->
<!-- requirement: images/Hadoop_ecosystem.png -->
<!-- requirement: images/MapReduce_Work_Structure.png -->
<!-- requirement: images/Hadoop_cluster.jpg -->
<!-- requirement: scripts/reducer.py -->
<!-- requirement: scripts/namenode-reset.sh -->

Sources: *Hadoop: The Definitive Guide, 4th Ed.*

## Big Data


Big data consists of large, growing files on the order of terabytes ($10^{12}$) and petabytes ($10^{15}$). Big data includes (among other things) transactional, sensor, and social data, which are typically unstructured and can exist in a variety of formats like text, audio, and visual files.

Over the past several decades, we have been able to store more data on our computers for far cheaper. However, the speed at which we can access data has not kept up with the exponential growth in data. Below is the amount of time it takes to access a random byte from memory.

- Random access memory (RAM): ~100ns
- Solid state disk (SSD): ~100μs
- Hard disk drive (HD): ~10ms

However, if you can read bytes sequentially from a hard drive, you can beat the random-access speed of RAM. For example, a disk that gives you 200MB/s sequential read gives a byte every 5ns!

Analyzing big data sets is impractical on one machine. Other challenges associated with big data include capturing, storing, searching, visualizing, and querying information. Distributed computing is the solution to challenges associated with big data.

## Distributed Computing


Instead of performing some task on a single machine, you can use a system of computers that are connected via a network. Network latencies (~1ms) aren't so bad compared to hard drives. This division of labor is called distributed computing.

Not all tasks, however, can be completed in a distributed computing framework. This occurs with inherently serial problems (an example of which is an algorithm that uses iterative numerical methods), where the result of one task relies on the outcome of a previous computation. So which tasks are suitable for distributed computing? The answer is any task where operations can take place independently. Problems that are particularly well suited for distributed computing are called **embarrassingly parallel**. Examples include 3D video rendering and growing random forests.

Distributing data and tasks amongst computers is a challenging problem. Fortunately we have a set of tools that keeps track of files, manages machine failures, and breaks up computations for us.

## Hadoop

A reliable, scalable platform for storage and analysis. Its three primary components are:
1. A fault tolerant, **distributed file system**
1. Fault tolerant **distributed processing**
1. A **resource manager**

![A Hadoop cluster](images/Hadoop_cluster.jpg)

A **cluster** is a collection of servers called **nodes** that communicate with each other over a network. Hadoop clusters consist of master and worker nodes.

The **master node** consists of a...

- **NameNode**: contains information on what data resides on what node
- **JobTracker**: receives jobs from a client, then delegates jobs and collects results from worker nodes

The **worker nodes** consist of a...

- **DataNode**: a subset of data
- **TaskTracker**: receives and processes jobs from the master node

**Job progression**:
1. Client applications submit jobs to the Job tracker.
1. The JobTracker talks to the NameNode to determine the location of the data
1. The JobTracker locates TaskTracker nodes with available slots at or near the data
1. The JobTracker submits the work to the chosen TaskTracker nodes.
1. The TaskTracker nodes are monitored. If they do not submit heartbeat signals often enough, they are deemed to have failed and the work is scheduled on a different TaskTracker.
1. A TaskTracker will notify the JobTracker when a task fails. The JobTracker decides what to do then: it may resubmit the job elsewhere, it may mark that specific record as something to avoid, and it may may even blacklist the TaskTracker as unreliable.
1. When the work is completed, the JobTracker updates its status.

## Hadoop File System (HDFS)


NameNodes and DataNodes comprise Hadoop's file system that is responsible for storing and replicating data. More specifically, HDFS is a file system designed for storing very large files with streaming data access patterns, running on clusters of commodity hardware. 

- *Streaming data access*: Write-once, read-many-times.
- *Distributed file system*: a file system that manages the storage across a network of machines. A complex problem! Network of machines
- *Fault Tolerant*: nodes can go down without suffering data loss.
- *Optimized for delivering a high throughput of data*, maybe at expense of latency. HBase is better for low-latency access

**Exercise: Get acquainted with HDFS:** Your JupyterHub pod already has a Hadoop 2.x version installed. This means that our machines, which is effectively a 16 GB RAM, quad-core computer in the cloud, can act like a single node running Hadoop. Hadoop can be run as a single Java process in non-distributed mode (i.e. standalone), or in pseudo-distributed mode where each Hadoop daemon (background process) runs in a separate process. That means on the box you can create and run Hadoop jobs that access the Hadoop distributed file system. This is useful for debugging.

At the command line of your pod, try out a few HDFS commands:
```bash
$ hadoop fs -ls /
Found 3 items
drwxrwxrwt   - hdfs supergroup          0 2015-03-27 13:42 /tmp
drwxr-xr-x   - hdfs supergroup          0 2015-03-25 15:35 /user
drwxr-xr-x   - hdfs supergroup          0 2015-03-25 15:35 /var

$ hadoop fs -du -h /user

$ hadoop fs -mkdir gutenberg

$ hadoop fs -copyFromLocal small_data/gutenberg/* gutenberg

$ hadoop fs -cat gutenberg/pg20417.txt | head


# run this mapreduce

$ hadoop jar /usr/lib/hadoop-mapreduce/hadoop-mapreduce-examples-2.*.jar grep gutenberg output 'strange'


# note that there is an output dir

$ hadoop fs -ls


# it contains a _SUCCESS flag and part-r-00000

$ hadoop fs -ls output


# here's the contents

$ hadoop fs -cat output/part-r-00000 | head
```
A full list of available commands available [here](http://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html).   You can also get information on various HDFS commands (and command options) with this:

```bash
$ hadoop fs -help
```

## MapReduce


Keeping state synchronized across multiple nodes is difficult.  Many distributed systems adopt a **functional programming** approach, with
- Pure functions (with no side effects)
- Immutable data

In December, 2004, Jeffery Dean and Sanjay Ghemawat of Google introduced [MapReduce](https://research.google.com/archive/mapreduce.html), a scheme for distributed computing based on two higher-order functions: **map** and **reduce**.  Both exist in Python:

**Map**: Parallelizable

In [1]:
def square(x):
    return x * x

#The list command is to make a readable output
#in python3 map and filter create generators
list(map(square, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

**Reduce**: Associative and commutative: $(a \star b) \star (c \star d)$

In [2]:
from functools import reduce

def add(x, y):
    return x + y

reduce(add, map(square, range(10)))

285

These functions are often specified with lambda, but that is only an issue of syntax.

In [ ]:
reduce(lambda x, y: x + y, map(lambda x: x * x, range(10)))

**Example**: Find the sum of squares of 1 million numbers

On 1 machine:
1. $10^6$ steps to square each number
1. $10^6$ steps to add all the numbers
1. $\mathcal{O}(n)$

On 1 million machines:
1. Map: 1 step to square all of the numbers
1. Reduce: 20 or $\log_2 (10^6)$ steps to add all the numbers
1. $\mathcal{O}(\log n)$

On Hadoop, the MapReduce engine consists of the JobTracker and TaskTrackers. Each TaskTracker has four slots available for a single map, combine, or reduce job.

## Example: Word Count


Serial approach: Read through a book from cover to cover, keeping a tally.

Parallel approach: Give pages to a bunch of people, have them tally each page, then sum the tallies.

_** `mapper.py` **_

```python
import sys

def read_input(file, encoding='utf-8'):
    for line in file:
        yield line.decode(encoding)
        
def write_line(line, encoding='utf-8'):
    sys.stdout.buffer.write((line + '\n').encode(encoding))

def main(separator='\t'):
    data = read_input(sys.stdin.buffer)
    for line in data:
        for word in line.split():
            write_line('%s%s%d' % (word, separator, 1))
            
if __name__ == "__main__":
    main()
```

** `reducer.py`** 

```python
from itertools import groupby
from operator import itemgetter
import sys

def read_input(file, encoding='utf-8'):
    for line in file:
        yield line.decode(encoding)

def write_line(line, encoding='utf-8'):
    sys.stdout.buffer.write((line + '\n').encode(encoding))

def main(separator='\t'):
    data = map(lambda line: line.split(separator, 1), read_input(sys.stdin.buffer))
    for current_word, group in groupby(data, itemgetter(0)):
        total_count = sum(int(count) for current_word, count in group)
        write_line("%s%s%d" % (current_word, separator, total_count))
        
if __name__ == "__main__":
    main()
```


### Approach 1: MapReduce with Unix pipes


From the command line:

```bash
cat small_data/gutenberg/* \
    | python scripts/mapper.py \
    | sort -k1,1 \
    | python scripts/reducer.py \
    | grep "^[a-z]" \
    | less
```

Or to run in parallel:

```bash
cat small_data/gutenberg/* \
    | parallel --pipe python scripts/mapper.py \
    | sort -k1,1 \
    | parallel --pipe python scripts/reducer.py \
    | grep "^[a-z]" \
    | less
```

(Note that the reducer doesn't quite work in parallel, since we haven't ensured that all copies of a key will go to same reducer.)


### Approach 2: Hadoop streaming

Unix pipes are constrained to only run on `localhost`.  When you want to run your code in a distributed manner, you will want to run them using Hadoop streaming:

```bash
export STREAMING_JAR=/usr/lib/hadoop-mapreduce/hadoop-streaming-*.jar

hadoop jar $STREAMING_JAR \
    -mapper ~/datacourse/mapreduce/scripts/mapper.py \
    -reducer ~/datacourse/mapreduce/scripts/reducer.py \
    -input gutenberg \
    -output output_gutenberg
```

The mapper task converts the inputs into lines and feeds the lines into the `stdin` of the process. 
The output from `stdout` of the mapper process is converted into a key/value pair. **By default, all characters up to the first tab character is the key and the rest of the line is the value.** This is customizable.

For full documentation, see: https://hadoop.apache.org/docs/r1.2.1/streaming.html

### MapReduce sequencing

![MapReduce Process Structure](images/MapReduce_Work_Structure.png)

The above figure can be found on Xiaocheng Zhang's blog.

1. **Input**: the input is a large file on a distributed file system stored as an input system and read according to a user-specified `InputFormat`.  For word count, it is a large text file.
1. **Splitting**: an Input Reader splits the file into chunks of 64 MB or 128 MB.  We think of each chunk as consisting of a collection of records which can be represented as key-value pairs `(k0, v0)`.  We think of the records as being unordered but keyed.  The splitting is done so that each chunk is also on a single machine.  While there can be multiple chunks on a single machine, it is useful to abstract things and assume that each chunk resides on its own machine.
1. **Mapping**: the user specifies a `map` function, which iterates through the records within a chunk.  It outputs key-value pairs `(k1, v1)` (which are written to the local disk).  It can output multiple key-value pairs per input record or none at all and the key and value need not have the same type as the input.  In our case, each input record is a line in a file (e.g. `"Deer Bear River"`) and there is an output record consisting of each word with the number one (e.g. `("Deer", 1)`), indicating that we saw the word `"Deer"` once.
1. **Combiner**: (skip this step the first time).  This runs a reduce right after mapping but before shuffling.  The **local aggregation** can reduce the amount of I/O in the shuffle phase, which is the most important bottleneck.
1. **Shuffle**: up until this point, all the computations have happened locally and no data has been exchanged between the servers.  During Shuffle, the key value pairs `(k1, v1)` are sent to a computer based on `k1` so that all pairs with the same key end up at the same computer (and are written to that computer's disk).  Think of `k1` as giving the *address* to which to send the data and `v1` as giving the data payload to deliver.  Again, which we can have multiple keys mapping to the same physical machine, it is useful to abstract this and assume each key residing on its own machine.
1. **Reduce**: the user specifies a `reduce` function, which is given pairs `(k1, iter(v1))` where `iter(v1)` is an iterator over the values.  An iterator is like a list except that there is no random access to elements and one can only iterate through its elements once in order.  Underneath the hood, all the data resides on disk and only the current record being iterated over is kept in memory.  The `reduce` function outputs key-value pairs `(k2, v2)` (which are written to the local disk).  Again, the number of values it outputs is independent of the number of inputs and they can be of different types.  In our case, it maps each `(k1, iter(v1))` to `(k1, sum(v1))`.
1. **Final Result**: The results at this point are written out to disk according to a user-specified `OutputFormat`.

**Finer points**:
1. The records in MapReduce are meant to be unordered.
1. MapReduce is fault tolerant, if one of the processes has an error, it will automatically be restarted.
1. You might think it's ridiculous to write `("Car", 1)` twice for the input record `"Car Car River"`.  We could have written `("Car", 2)`.  How much memory would it require to keep track of this?  In general, we want the amount of memory used to be constant in the number of input records - this is what makes MapReduce scalable.  That said, you should look into the `combiner` function.  The typical use case is to run reduce on all the values `(k1, iter(v1))` which happen to be on the same mapper computer as a speed up.  To find out more, look at the [Apache tutorial](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html).  What does this save you, CPU time, RAM, or Network Load?
1. For more information, checkout [Apache](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html).


## YARN

**YARN** ("Yet Another Resource Negotiator"): a cluster resource management system introduced in Hadoop 2.0 (c. February 2013) which allows any distributed program (not just MapReduce) to run on data in a Hadoop cluster.
Critical for allowing new processing models in Hadoop, i.e. the flourishing of the Hadoop ecosystem. 


### The different processing patterns that work with Hadoop

- **Batch processing**: This is what Hadoop was designed to do.  You know what you want to do, write the job, and run it over large amounts of data.
- **Interactive SQL**: By dispensing with MapReduce and using a distributed query engine that uses dedicated “always on” daemons (like `Impala`) or container reuse (like `Hive` on `Tez`), it’s possible to achieve low-latency responses for SQL queries on Hadoop while still scaling up to large(-ish) data set sizes.  
- **Iterative processing**: Many algorithms - such as those in machine learning - are iterative in nature, so it’s much more efficient to hold each intermediate working set in memory, compared to loading from disk on each iteration. The architecture of MapReduce does not allow this, but it’s straightforward with *Spark*, for example, and it enables a highly exploratory style of working with data sets.  
- **Stream processing**: Streaming systems like `Storm`, `Spark Streaming`, or `Samza` make it possible to run real-time, distributed computations on unbounded streams of data and emit results to Hadoop storage or external systems.  

## The Hadoop ecosystem

![A Hadoop stack](images/Hadoop_ecosystem.png) 


## Comparing Hadoop with alternatives

- It's cheap: Hadoop itself is free; commodity clusters are inexpensive
- vs. Relational Database Management Systems
>  MapReduce is a good fit for problems that need to analyze the whole data set in a batch fashion, particularly for ad hoc analysis. An RDBMS is good for point queries or updates, where the data set has been indexed to deliver low-latency retrieval and update times of a relatively small amount of data. *MapReduce suits applications where the data is written once and read many times*, whereas a relational database is good for data sets that are continually updated.

- vs. Grid Computing or High Performance Computing
> Broadly, the approach in high performance computing is to distribute the work across a cluster of machines, which access a shared file system, hosted by a storage area network (SAN). *This works well for predominantly compute-intensive jobs*, but it becomes a problem when nodes need to access larger data volumes (hundreds of gigabytes, the point at which Hadoop really starts to shine), since the network bandwidth is the bottleneck and compute nodes become idle.
Hadoop tries to co-locate the data with the compute nodes, so data access is fast because it is local. This feature, known as *data locality*, is at the heart of data processing in Hadoop and is the reason for its good performance. 

- Programming in Hadoop
> MPI gives great control to programmers, but it requires that they explicitly handle the mechanics of the data flow, exposed via **low-level** C routines and constructs such as sockets, as well as the higher-level algorithms for the analyses. Processing in Hadoop operates only at the **higher level**: the programmer thinks in terms of the data model (such as key-value pairs for MapReduce), while the data flow remains implicit.


### So, what's different about programming around big data?

- Performance considerations: get rid of straggler tasks (e.g. resulting from [skew](http://nuage.cs.washington.edu/pubs/opencirrus2011.pdf)). E.g.: When using MapReduce, optimize the number of reducers; add a `combiner`.
- Hardware configuring and tuning. 
- Monitoring the progress of your jobs
- Often: opaque error messages.


### If Hadoop is not working on your system, you might need to restart


Try running the `./scripts/namenode-reset.sh` script inside the `mapreduce` directory. Alternatively, you can run the following commands: 

```bash
sudo su root  # become the root user
echo JAVA_HOME=/usr/lib/jvm/java-8-oracle-amd64 >> /etc/environment
echo "export JAVA_HOME=/usr/lib/jvm/java-8-oracle-amd64" >> /etc/alternatives/hadoop-conf/hadoop-env.sh


# run command-D to get out of root mode


# Start HDFS

sudo service hadoop-hdfs-datanode start
sudo service hadoop-hdfs-namenode init
sudo service hadoop-hdfs-namenode start

sudo -u hdfs hadoop fs -mkdir /user/
sudo -u hdfs hadoop fs -mkdir /user/jovyan
sudo -u hdfs hadoop fs -chown jovyan /user/jovyan


# Check to make sure everything works now  

hadoop fs -put small_data/fha_by_tract.csv /user/jovyan
```

If you get an error informing you that the Hadoop NameNode is dead, [these instructions](http://kshitish-bigdata.blogspot.com/2015/02/hadoop-namenode-is-dead-and-pid-file.html) may help you get it running again.

### Exit Tickets

1. Can/should the traveling salesman problem be solved using the MapReduce paradigm?
1. At which points in MapReduce are information transmitted over the network?
1. From the perspective of a worker node doing mapping, describe the format of data it sees as input and what it outputs. Same for reducing. 

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*